In [1]:
import os
import sys
import tensorflow as tf
import keras

# we need to append the root directory to the path to import data.py\n",
sys.path.append('..')
import data
import utils
import model

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir_images'] = os.path.join('..', params['data_dir_images'])
params['data_dir'] = os.path.join('..', params['data_dir'])

## Model Summary

In [4]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
architecture = model.MobilNet_Architecture(
    width_multiplier=params['width_multiplier'],
    depth_multiplier=params['depth_multiplier'],
    num_classes=params['num_classes'],
    dropout_rate=params['dropout_rate'])

x = architecture(inputs, training=False)
architecture.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_27 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
activation_28 (Activation)   (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_separable_convolut (None, 112, 112, 64)      3840      
_________________________________________________________________
depthwise_separable_convolut (None, 56, 56, 128)       13824     
_________________________________________________________________
depthwise_separable_convolut (None, 56, 56, 128)       35200     
_________________________________________________________________
depthwise_separable_convolut (None, 28, 28, 256)       52224     
__________

In [11]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
architecture = model.MobilNet_Architecture_Tiny(
    width_multiplier=params['width_multiplier'],
    depth_multiplier=params['depth_multiplier'],
    num_classes=params['num_classes'],
    dropout_rate=params['dropout_rate'])

x = architecture(inputs, training=False)
architecture.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_111 (Bat (None, 112, 112, 32)      128       
_________________________________________________________________
activation_112 (Activation)  (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_separable_convolut (None, 56, 56, 128)       11488     
_________________________________________________________________
depthwise_separable_convolut (None, 28, 28, 256)       52224     
_________________________________________________________________
depthwise_separable_convolut (None, 14, 14, 512)       202752    
_________________________________________________________________
depthwise_separable_convolut (None, 14, 14, 512)       534016    
__________